In [1]:
import os.path
import requests
from bs4 import BeautifulSoup
import re
import json
import html
import pandas as pd
import datetime
import time
import random

from fake_useragent import UserAgent
from tqdm import tqdm

In [2]:
base_link = 'https://www.xe.gr/property/results?transaction_name=rent&item_type=re_residence&geo_place_ids[]=ChIJ8UNwBh-9oRQR3Y1mdkU1Nic&page='

In [3]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}
response = requests.get(base_link + '1', headers=user_agent)
if response.status_code == 200:
    print("Success")
else:
     print("Request failed")   

Success


In [4]:
soup = BeautifulSoup(response.content, 'html.parser')
# page_json = re.search(r'(?<=data-json-data=\').*(?=\' data-language)', str(soup)).group(0)
page_json = re.search(r'(?<=data-json-data=.).*(?=. data-language)', str(soup)).group(0) # sometimes json regex needs "" isntead of ''
my_json_content = json.loads(page_json)
page_count = my_json_content['selected_values']['pagination']['total_pages']
page_count

174

In [5]:
failed_pages = 0
json_content_list = []
for page_number in tqdm(range(page_count)):
# for page_number in tqdm(range(151, 172)): # temp fix to get pages after a ban
    page_link = base_link + str(page_number + 1)
    user_agent = {'User-agent': ua.random}
    response = requests.get(page_link, headers=user_agent)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
#         str_json = re.search(r'(?<=data-json-data=\').*(?=\' data-language)', str(soup)).group(0)
        str_json = re.search(r'(?<=data-json-data=.).*(?=. data-language)', str(soup)).group(0) # sometimes json regex needs "" isntead of ''
#         my_json_content = json.loads(str_json)
        my_json_content = json.loads(html.unescape(str_json))  # sometimes str_json comes in escaped format (e.g. "{&quot;results&quot;}"), this gets resolved by using html.unescape
        json_content_list.append(my_json_content)
#         time.sleep(0.6)
        time.sleep(random.uniform(0.9, 1.5))
    else:
        failed_pages +=1
        next

print(f"{page_count - failed_pages}/{page_count} successes")

100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [01:30<00:00,  1.92it/s]

13/174 successes


In [6]:
def convert_json_to_df(page_json):
    page_apartments = page_json['results']
    return pd.DataFrame(page_apartments)

df_list = list(map(convert_json_to_df, json_content_list))

In [7]:
master_df = pd.concat(df_list)
master_df['extraction_date'] = pd.to_datetime(datetime.datetime.today().date())

In [8]:
master_df.to_csv(os.path.join('Data', 'Raw', f'xe_{datetime.datetime.today().strftime("%Y%m%d")}.csv'), encoding='utf-8-sig', index=False)